### Single Run with Priority

This notebook is developed based on single run for multiple containers. One dredging vessel along with one seagoing vessel serve the same berth area.
The aim of this notebook is to find out how the single run and vessels log change when
* case 1: the dredging vessel is prior
* case 2: the seagoing vessel is prior

In [101]:
import networkx as nx  
import shapely.geometry
import pyproj


import datetime, time
import simpy
import openclsim
import openclsim.core as core
import openclsim.model as model

import pandas as pd

import matplotlib.pyplot as plt

In [102]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

In [103]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasMultiContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.MultiContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.Priority,
        core.LoadingFunction,
        core.UnloadingFunction
    ),
    {},
)

In [104]:
# three locations (offshore, berth, and dump) are defined to allow the vessels pass the nodes back and forth.
# based on the mixins, more attributes are defined for each location.
locations = []

location_offshore = shapely.geometry.Point(3.637610458072877, 52.10701582514347)
data_offshore = {"env": my_env,
                 "name": "offshore",
                 "geometry": location_offshore,
                 "store_capacity": 1,
                 "initials": [
                                {"id": "Container", "level": 0, "capacity": 0},
                                {"id": "Soil", "level":  0, "capacity":  0},
                              ],
                }
locations.append(Site(**data_offshore))
offshore = Site(**data_offshore)

location_berth = shapely.geometry.Point(4.062705498083785,51.95909715255291)
data_berth = {"env": my_env,
              "name": "berth",
              "geometry": location_berth,
              "store_capacity": 200,
              "initials": [
                            {"id": "Container", "level": 50, "capacity": 100},
                            {"id": "Soil", "level":  80, "capacity":  100},
                            ],
             }
locations.append(Site(**data_berth))
berth = Site(**data_berth)

location_dump = shapely.geometry.Point(4.1795229742823,52.11590591837503)
data_dump = {"env": my_env,
             "name": "dump",
             "geometry": location_dump,
             "store_capacity": 100,
             "initials": [
                            {"id": "Container", "level": 0, "capacity": 0},
                            {"id": "Soil", "level":  0, "capacity":  100},
                            ],
            }
locations.append(Site(**data_dump))
dump = Site(**data_dump)

In [105]:
# prepare input data for dredging_vessel
data_dredging_vessel = {"env": my_env,
                        "name": "vessel01",
                        "geometry": location_dump, 
                        "loading_rate": 2,
                        "unloading_rate": 4,
                        "store_capacity": 7,
                        "initials": [
                                      {"id": "Container", "level": 0, "capacity": 0},
                                      {"id": "Soil", "level":  0, "capacity":  7},
                                    ],
                        "compute_v": lambda x: 10,
                        "priority": 1
                      }
# instantiate dredging_vessel
dredging_vessel = TransportProcessingResource(**data_dredging_vessel)

# prepare input data for seagoing_vessel
data_seagoing_vessel = {"env": my_env,
                        "name": "vessel01",
                        "geometry": location_offshore, 
                        "loading_rate": 3,
                        "unloading_rate": 5,
                        "store_capacity": 80,
                        "initials": [
                                      {"id": "Container", "level": 0, "capacity": 0},
                                      {"id": "Soil", "level":  0, "capacity":  80},
                                    ],
                        "compute_v": lambda x: 5,
                        "priority": 2
                        }
# instantiate dredging_vessel
seagoing_vessel = TransportProcessingResource(**data_seagoing_vessel)

In [106]:
# initialise registry
registry = {}

In [107]:
requested_resources={}

# create a list of the sub processes of dredging vessel
sub_processes_dredging = [
    model.MoveActivity(
        env=my_env,
        name="sailing empty dredging vessel",
        registry=registry,
        mover=dredging_vessel,
        destination=berth,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="loading soil",
        registry=registry,
        processor=dredging_vessel,
        origin=berth,
        destination=dredging_vessel,
        amount=7,
        id_="soil",
        duration=20,
        requested_resources=requested_resources,
    ),
    model.MoveActivity(
        env=my_env,
        name="sailing full dredging vessel",
        registry=registry,
        mover=dredging_vessel,
        destination=dump,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="unloading",
        registry=registry,
        processor=dredging_vessel,
        origin=dredging_vessel,
        destination=dump,
        amount=7,
        duration=10,
        requested_resources=requested_resources,
    ),
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[dredging_vessel],
    ),
]


activities_dredging_vessel = model.WhileActivity(
    env=my_env,
    name="while",
    registry=registry,
    sub_processes=[model.SequentialActivity(
        env=my_env,
        name="sequential_activity_dredging_vessel",
        registry=registry,
        sub_processes=sub_processes_dredging
    )],
    condition_event=[{"type": "soil", "concept": dump, "state": "full", "id_":"default_reservations"}],
)

In [108]:
requested_resources={}

# create a list of the sub processes of seagoing vessel
sub_processes_seagoing = [
    model.MoveActivity(
        env=my_env,
        name="sailing empty seagoing vessel",
        registry=registry,
        mover=seagoing_vessel,
        destination=berth,
    ),
    model.ShiftAmountActivity(
        env=my_env,
        name="loading container",
        registry=registry,
        processor=seagoing_vessel,
        origin=berth,
        destination=seagoing_vessel,
        amount=80,
        id_="container",
        duration=50,
        requested_resources=requested_resources,
    ),
    model.MoveActivity(
        env=my_env,
        name="sailing full seagoing vessel",
        registry=registry,
        mover=seagoing_vessel,
        destination=offshore,
    ),
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[seagoing_vessel],
    ),
]


activities_seagoing_vessel = model.WhileActivity(
    env=my_env,
    name="while",
    registry=registry,
    sub_processes=[model.SequentialActivity(
        env=my_env,
        name="sequential_activity_seagoing_vessel",
        registry=registry,
        sub_processes=sub_processes_seagoing
    )],
    condition_event=[{"type": "container", "concept": offshore, "state": "full", "id_":"default_reservations"}],
)

In [109]:
simulation_process = openclsim.model.ParallelActivity(
    env=my_env,
    name="all activities",
    registry=registry,
    sub_processes=[activities_dredging_vessel, activities_seagoing_vessel]
)

In [110]:
model.register_processes([simulation_process])

AssertionError: 